In [8]:
import numpy as np
import pandas as pd
import ast

In [9]:
data_test = pd.read_csv('./intermediate_files/test.csv')
data_test.head()

,piece,beats,pitches,beat_pitch
0,../abc/Polska_i_Dm__cafbff.abc,"[[1, 3, 5, 6, 7, 8, 9, 11], [1, 3, 4, 5, 7, 9,...","[[74, 76, 77, 76, 77, 79, 81, 77], [81, 77, 81...","[[(1, 74), (3, 76), (5, 77), (6, 76), (7, 77),..."
1,../abc/Glada_Ryd_1cb655.abc,"[[1, 2, 3, 4, 5, 6, 7, 8, 9, 12], [1, 2, 3, 4,...","[[76, 81, 80, 81, 76, 81, 80, 81, 76, 76], [78...","[[(1, 76), (2, 81), (3, 80), (4, 81), (5, 76),..."
2,../abc/_Polska_efter_Petter_Dufva__58c596.abc,"[[1, 3, 5, 7, 9], [1, 3, 5, 7, 9, 11], [1, 3, ...","[[74, 73, 74, 78, 69], [67, 66, 64, 67, 66, 62...","[[(1, 74), (3, 73), (5, 74), (7, 78), (9, 69)]..."
3,../abc/Pollonesse_183223_93b57b.abc,"[[1, 2, 3, 4, 5, 7, 8, 9, 11, 12], [1, 3, 4, 5...","[[74, 76, 78, 76, 74, 74, 76, 73, 73, 74], [71...","[[(1, 74), (2, 76), (3, 78), (4, 76), (5, 74),..."
4,../abc/Polonäs_i_Gm_efter_Baumgarten_e8ce25.abc,"[[11], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]...","[[69], [70, 72, 74, 72, 72, 70, 69, 70, 67, 66...","[[(11, 69)], [(1, 70), (2, 72), (3, 74), (4, 7..."


In [10]:
data_test['beat_pitch'] = data_test['beat_pitch'].apply(ast.literal_eval)
print(data_test['beat_pitch'])

0      [[(1, 74), (3, 76), (5, 77), (6, 76), (7, 77),...
1      [[(1, 76), (2, 81), (3, 80), (4, 81), (5, 76),...
2      [[(1, 74), (3, 73), (5, 74), (7, 78), (9, 69)]...
3      [[(1, 74), (2, 76), (3, 78), (4, 76), (5, 74),...
4      [[(11, 69)], [(1, 70), (2, 72), (3, 74), (4, 7...
                             ...                        
116    [[(11, 72), (12, 70)], [(1, 69), (2, 72), (3, ...
117    [[(1, 65), (2, 69), (3, 72), (7, 70), (8, 74),...
118    [[(1, 76), (3, 72), (5, 69), (6, 71), (7, 72),...
119    [[(1, 74), (3, 79), (5, 79), (7, 78), (9, 78),...
120    [[(11, 69)], [(1, 74), (4, 78), (5, 81), (7, 8...
Name: beat_pitch, Length: 121, dtype: object


In [26]:
def format_transition_1(piece):
    # Transfer the beat_pitch format to 12-position 1-12 scale format
    # arg: 1.piece: one piece of music in beat_pitch format
    # return: 1.re: the transfer format of the piece, re is in shape of number_of_bars * 12
    re=np.zeros((len(piece),12))
    for i in range(len(piece)):
        bar=piece[i]    
        for j in range(len(bar)):
            midi_num=bar[j][1]%12
        if(bar[j][1]%12==0):
            midi_num=12
        re[i][bar[j][0]-1]=midi_num
    return re  

In [27]:
def format_transition_2(piece):
    # Transfer the beat_pitch format to 12-position 61-72 scale format
    # arg: 1.piece: one piece of music in beat_pitch format
    # return: 1.re: the transfer format of the piece, re is in shape of number_of_bars * 12
    re=np.zeros((len(piece),12))
    for i in range(len(piece)):
        bar=piece[i]    
        for j in range(len(bar)):
            midi_num=bar[j][1]%12+60
        if(bar[j][1]%12==0):
            midi_num=72
        re[i][bar[j][0]-1]=midi_num
    return re  

In [30]:
def compare(generated_piece,test_piece):
    # arg: 1.generated_piece: the piece generated by ourselves. 2. test_piece: one piece in test set.
    # retrun 1.difference: the difference between two input pieces, the larger the more different of the two pieces
    
    difference=0
    generated_piece_re=format_transition_1(generated_piece)
    test_piece_re=format_transition_1(test_piece)
    compare_bar_num=len(generated_piece) if len(generated_piece)<len(test_piece) else len(test_piece) # select the piece which has the smaller number of bars
    for i in range(compare_bar_num):
        for j in range(12):
            difference=difference+abs(generated_piece_re[i][j]-test_piece_re[i][j])
    return difference

In [33]:
# An example
data_train = pd.read_csv('./intermediate_files/train.csv')
data_train['beat_pitch'] = data_train['beat_pitch'].apply(ast.literal_eval)
generated_piece=data_train['beat_pitch'][88]

min_difference=0
for i in range(len(data_test['beat_pitch'])):
    difference=compare(generated_piece,data_test['beat_pitch'][i])
    if(difference<min_difference):
        min_difference=difference
print(min_difference)

0
